In [50]:
import pandas as pd
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [60]:
images_list=sorted(glob("./processed_data/Nov_21/3_reduced/*.png"))
camera_data_clean=[x.split("/")[-1].split(".")[0] for x in images_list]
camera_data_clean=[x.split("_")[0]+"."+x.split("_")[1] for x in camera_data_clean]
camera_timestamps=np.array(camera_data_clean,np.float128)

In [41]:
camera_tared=camera_timestamps-camera_timestamps[0]

In [42]:
camera_tared

array([ 0.       ,  2.017643 ,  4.0694752,  6.0797034,  8.0189543,
        9.9922037, 11.0641108, 13.0056505], dtype=float128)

In [43]:
df=pd.read_csv("./processed_data/Nov_21/3/dataset3_2023_12_13_11_07_58_sync.csv")

In [44]:
times=np.array(df["Timestamp"]-df["Timestamp"][0],np.float128)

In [45]:
times_tared=times/np.power(10,9)

In [46]:
times_tared

array([ 0.        ,  0.74299908,  1.49830794,  1.71865463,  2.45717668,
        3.20690107,  3.9531486 ,  4.14329672,  4.8815434 ,  5.1729176 ,
        5.92322207,  6.1311903 ,  6.89190507,  7.09300303,  7.83300018,
        8.58424592,  9.33774281, 10.0872364 , 10.82265973, 11.56315661,
       12.27023721, 13.00342584, 13.73338699, 14.47285771, 15.24516916,
       15.41427469, 16.16637826, 16.92284179, 17.67636418, 18.41923666,
       19.17212462, 19.91271401, 20.65270019, 21.44306231, 22.12305284,
       22.88344717, 23.64136481, 24.40175104, 25.15938187, 25.92578459,
       25.93163347, 26.10188484, 26.11190009, 26.1216526 , 26.13159037,
       33.36232972, 34.12545872, 34.82149816, 35.0678246 , 35.83175421,
       36.08288693, 36.79230022, 37.56986976, 38.36414027, 39.09737039,
       39.82639837, 40.61181021, 41.329247  , 42.09228158, 42.33197498,
       43.08453274, 43.86500192, 44.80145788, 45.31142521, 46.04366112,
       46.78954101, 48.21237373, 48.22126389, 49.02980471, 49.76

In [47]:
lst=[]
for x in camera_tared:
    lst.append(np.argmin(np.abs(times_tared-x)))

In [66]:
rotation_matrices = [np.eye(3)] * len(lst)  # Assuming the same rotation matrix for all images
translation_vectors = [np.array([df.iloc[i]["Pose_X"],df.iloc[i]["Pose_Y"],df.iloc[i]["Pose_Z"]]) for i in lst]  # Replace with actual translation vectors

def construct_extrinsic_matrix(rotation_matrix, translation_vector):
    extrinsic_matrix = np.hstack((rotation_matrix, translation_vector.reshape(-1, 1)))
    extrinsic_matrix = np.vstack((extrinsic_matrix, [0, 0, 0, 1]))
    return extrinsic_matrix

def align_images(images, extrinsic_matrices):
    aligned_images = []
    reference_matrix = np.eye(4)

    for i in range(len(images)):
        print("in")
        # Compute the relative transformation between the current image and the reference image
        relative_matrix = np.linalg.inv(extrinsic_matrices[i]) @ reference_matrix
        print(relative_matrix[:3, :])

        # Apply the relative transformation to the current image
        aligned_image = cv2.warpPerspective(images[i], relative_matrix[:3, :], (images[i].shape[1], images[i].shape[0]))

        aligned_images.append(aligned_image)

    return aligned_images

def stitch_images(images):
    # Create a stitcher object
    stitcher = cv2.Stitcher_create()

    # Set registration parameters if needed
    #stitcher.setRegistrationResol(0.6)
    #stitcher.setWaveCorrection(False)
    #stitcher.setFeaturesMatcher(cv2.detail_BestOf2NearestMatcher_create(False, 0.6))

    # Extract extrinsic matrices for each image
    extrinsic_matrices = [construct_extrinsic_matrix(rotation_matrix, translation_vector)
                          for rotation_matrix, translation_vector in zip(rotation_matrices, translation_vectors)]

    # Align images using extrinsic matrices
    aligned_images = align_images(images, extrinsic_matrices)

    # Stitch aligned images
    status, panorama = stitcher.stitch(aligned_images)

    if status == cv2.Stitcher_OK:
        plt.imshow("Panorama", panorama)
    else:
        print("Stitching failed")

# Example usage:
# Assume you have the following lists for each image



In [67]:


# Load images
images = [cv2.imread(img_path) for img_path in images_list]

# Stitch images
stitch_images(images)

in
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.49997058e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.02275409e-03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 1.12533375e-12]]


error: OpenCV(4.2.0) ../modules/imgproc/src/imgwarp.cpp:3167: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'


In [59]:
images

[array([[[145, 151, 127],
         [145, 151, 127],
         [145, 152, 126],
         ...,
         [146, 166, 149],
         [144, 166, 149],
         [144, 166, 149]],
 
        [[144, 151, 123],
         [145, 153, 124],
         [144, 148, 122],
         ...,
         [146, 166, 147],
         [144, 166, 149],
         [144, 166, 149]],
 
        [[132, 138, 107],
         [134, 140, 110],
         [138, 144, 113],
         ...,
         [146, 166, 147],
         [144, 166, 149],
         [144, 166, 149]],
 
        ...,
 
        [[159, 157, 121],
         [159, 157, 121],
         [156, 156, 122],
         ...,
         [158, 165, 130],
         [159, 166, 131],
         [158, 165, 130]],
 
        [[157, 158, 121],
         [157, 158, 121],
         [157, 157, 123],
         ...,
         [158, 165, 130],
         [159, 166, 131],
         [158, 165, 130]],
 
        [[160, 158, 122],
         [160, 158, 122],
         [158, 157, 125],
         ...,
         [160, 165, 130],
  